# Hands-on Salmon

We will define a generative model that simulates fragments (and their corresponding reads) from a known transcriptome

In [2]:
import numpy as np

In [ ]:
def simulate_rna_seq_reads(
    transcripts: dict[str, str],
    alpha: list[int],
    num_reads: int = 10_000,
    frag_len_dist=None,
    seq_bias_5prime=None,
    seq_bias_3prime=None,
    gc_bias=None,
    rng=None,
):
    """
    Simulate reads according to a generative model similar to Salmon.

    $$
    T = {t_1, t_2, \ldots, t_M}.
    $$

    Args:
        transcripts: A dictionary `{ transcript_id : transcript_sequence }`.
        alpha: Transcript abundance proportions (must sum to 1).
        num_reads : int
            Total number of reads (fragments) to simulate.
        frag_len_dist : function -> int
            A function that samples from the chosen fragment length distribution.
            E.g. a lambda that calls np.random.  If None, we’ll assume a simple fixed length.
        seq_bias_5prime : function(str) -> float, optional
            A function that, given the 5'-end context, returns a multiplicative bias.
            If None, no 5'-end bias is applied.
        seq_bias_3prime : function(str) -> float, optional
            A function that, given the 3'-end context, returns a multiplicative bias.
            If None, no 3'-end bias is applied.
        gc_bias : function(float) -> float, optional
            A function that, given GC content of a fragment, returns a multiplicative bias.
        rng : np.random.Generator, optional
            Random number generator for reproducibility. If None, a new default is created.

    Returns
    -------
    reads : list
        A list of (transcript_id, start_pos, fragment_seq) for each simulated fragment.
    """
    if rng is None:
        rng = np.random.default_rng()

    # Gather transcript IDs and sequences
    tx_ids: list[str] = list(transcripts.keys())
    tx_seqs = list(transcripts.values())
    M = len(tx_ids)

    # Basic checks
    alpha = np.array(alpha, dtype=float)
    alpha /= alpha.sum()  # Ensure it sums to 1

    # Default fragment length distribution if none is given
    if frag_len_dist is None:
        # Suppose we always sample fragments of length ~ 200 plus some noise
        def frag_len_dist():
            return max(25, int(rng.normal(200, 30)))  # just an example

    reads = []

    # Precompute transcript lengths
    tx_lengths = np.array([len(seq) for seq in tx_seqs])

    # For each read to simulate:
    for _ in range(num_reads):
        # 1. Choose a transcript index i ~ Multinomial(alpha)
        i = rng.choice(M, p=alpha)

        # 2. Sample fragment length
        L = frag_len_dist()

        # 3. Select a start position
        #    We'll ensure there's room for a fragment of length L
        max_start = max(0, tx_lengths[i] - L)
        if max_start == 0:
            # If the transcript is too short, skip or try smaller L
            # In a real simulator, you'd handle corner cases more gracefully.
            continue
        start = rng.integers(low=0, high=max_start + 1)

        # 4. Extract the fragment sequence from the transcript
        frag_seq = tx_seqs[i][start : start + L]

        # 5. Compute (multiplicative) bias from 5' sequence context
        bias_factor = 1.0
        kmer_len = 5  # e.g. consider 5bp context
        if seq_bias_5prime:
            # pad if near start
            context_5prime = tx_seqs[i][max(0, start - kmer_len) : start]
            bias_factor *= seq_bias_5prime(context_5prime)

        # 6. Compute (multiplicative) bias from 3' sequence context
        if seq_bias_3prime:
            # pad if near end
            context_3prime = tx_seqs[i][(start + L) : (start + L + kmer_len)]
            bias_factor *= seq_bias_3prime(context_3prime)

        # 7. Compute GC bias (entire fragment)
        if gc_bias:
            gc_frac = (frag_seq.count('G') + frag_seq.count('C')) / float(len(frag_seq))
            bias_factor *= gc_bias(gc_frac)

        # 8. Accept or reject fragment according to bias_factor.
        #    A typical generative approach would weight transcripts by bias_factor,
        #    but here we’ll do a simple “probabilistic acceptance” approach:
        #    e.g., if bias_factor < 0.5, we might reject half the time, etc.
        #    Or we can simply store the factor to re-weigh probabilities later.
        if rng.random() < bias_factor:
            # Store the read
            reads.append((tx_ids[i], start, frag_seq))

    return reads

<>:11: SyntaxWarning: invalid escape sequence '\l'
<>:11: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipykernel_19556/4027555006.py:11: SyntaxWarning: invalid escape sequence '\l'
  """


In [4]:

# Suppose we have a tiny transcriptome of 2 transcripts
transcripts_demo = {
    "txA": "AGCGTACGTACTGACGTAGCTAGCTAGCGTACCGTTAGCA"*3,
    "txB": "CGTACGGGTTTACGATCGATCGTAGCGTACGATCATTAACC"*2,
}

# 50:50 initial transcript proportions
alpha_demo = [0.5, 0.5]

# Example biases (very rough placeholders)
def seq_bias_5prime_example(seq5):
    # if seq5 ends with "G" often, let's prefer it
    return 1.2 if seq5.endswith("G") else 1.0

def seq_bias_3prime_example(seq3):
    return 1.1 if seq3.startswith("A") else 1.0

def gc_bias_example(gc_frac):
    # Suppose we strongly favor mid-range GC
    return np.exp(-5 * (gc_frac - 0.5)**2)

simulated_reads = simulate_rna_seq_reads(
    transcripts_demo,
    alpha_demo,
    num_reads=1000,
    seq_bias_5prime=seq_bias_5prime_example,
    seq_bias_3prime=seq_bias_3prime_example,
    gc_bias=gc_bias_example
)
print(simulated_reads)
print("Simulated {} reads.".format(len(simulated_reads)))

[]
Simulated 0 reads.


Next, we want to perform abundance estimation (quantification).
We can define an 'assignment matrix' X of shape (N, M), where:

   X[i,k] = Probability that read i originated from transcript k.

Given:
 - A set of N reads R = {r1, r2, ..., rN}
 - M transcripts T = {t1, t2, ..., tM}
 - Some auxiliary model p(r | t), e.g. a computed alignment likelihood 
   or a simpler approximate match function that reflects bias.

We also have abundance parameters alpha_k >= 0, with sum(alpha_k) = 1.

A typical maximum-likelihood approach (like in Salmon or an EM algorithm) 
tries to maximize p(R | alpha, bias-params). That is often solved via 
an EM loop, or by direct optimization. Below we give a minimal demonstration 
using SciPy’s "minimize" or "least_squares" to optimize alpha and X subject to constraints.

In [ ]:

import numpy as np
from scipy.optimize import minimize

def assignment_likelihood(read_likelihoods, alpha):
    """
    Negative log-likelihood of the reads given alpha, 
    ignoring normalizing constants. 

    read_likelihoods is shape (N, M),
      read_likelihoods[i,k] = p(r_i | transcript k, bias-params).

    alpha is shape (M,) with sum(alpha)=1, alpha >= 0.

    The probability that read i came from transcript k is:
      X[i,k] = alpha[k] * read_likelihoods[i,k]
               / sum_j [alpha[j] * read_likelihoods[i,j]]

    The negative log-likelihood is 
      -sum_i log( sum_k [ alpha[k] * read_likelihoods[i,k] ] ).
    """
    # add small eps to avoid log(0)
    eps = 1e-15
    p_reads = np.sum(alpha * read_likelihoods, axis=1) + eps
    return -np.sum(np.log(p_reads))

def constraint_sum_to_one(alpha):
    # sum(alpha) - 1 = 0 as a constraint
    return np.sum(alpha) - 1.0

def abundance_estimation(read_likelihoods, init_alpha=None):
    """
    Use SciPy's minimize to optimize the negative log-likelihood
    subject to sum(alpha)=1, alpha>=0.
    """
    N, M = read_likelihoods.shape
    if init_alpha is None:
        init_alpha = np.ones(M) / M  # uniform init

    # We'll use SLSQP to handle constraints
    cons = ({'type': 'eq',  'fun': constraint_sum_to_one})
    bnds = [(0.0, 1.0) for _ in range(M)]

    # Objective for minimize
    def objective(alpha):
        return assignment_likelihood(read_likelihoods, alpha)

    res = minimize(
        objective,
        init_alpha,
        method='SLSQP',
        bounds=bnds,
        constraints=cons,
        options={"maxiter": 100, "ftol": 1e-7},
    )
    return res

In [ ]:
# Suppose we have 3 transcripts and 10 reads
# read_lik[i,k] = p(r_i | t_k), which you might get from alignment or
# a bias model. Here we just make up some numbers.
read_lik = np.array([
    [0.1,  0.01, 0.05],
    [0.2,  0.10, 0.02],
    [0.05, 0.06, 0.02],
    [0.10, 0.02, 0.09],
    [0.15, 0.04, 0.04],
    [0.12, 0.07, 0.01],
    [0.05, 0.02, 0.01],
    [0.03, 0.07, 0.08],
    [0.06, 0.03, 0.15],
    [0.01, 0.10, 0.05],
])

# Run the optimization
result = abundance_estimation(read_lik)

if result.success:
    print("Optimized alpha:", result.x)
    # We can reconstruct assignment matrix if needed:
    # X[i, k] = alpha[k] * read_likelihoods[i,k] / sum_j[ alpha[j]*read_likelihoods[i,j] ]
    alpha_opt = result.x
    X = alpha_opt * read_lik
    denom = X.sum(axis=1, keepdims=True)
    X /= denom
    print("Assignment matrix (first few reads):\n", X[:5])
else:
    print("Optimization failed:", result.message)



1. **Bias Modeling**  
   In the example generative function simulate_rna_seq_reads, we showed how you might incorporate 5′ context, 3′ context, and GC content biases multiplicatively when deciding whether or not to “accept” or “reject” a fragment for output. Salmon uses a more nuanced approach by weighting transcript likelihoods by the learned bias parameters and then performing inference over all reads collectively citeturn0file0.  

2. **EM vs. Direct Constrained Optimization**  
   In Salmon, the assignment matrix X (the conditional probability that read i comes from transcript k) is not stored explicitly; instead, it is computed in each E-step. However, the simple demonstration above uses scipy.optimize.minimize to show how you could do direct likelihood-based estimation subject to constraints (sum(alpha)=1, alpha ≥ 0). An alternative is to implement the EM loop manually, especially if you want to handle large data sets more efficiently (by collapsing equivalent reads, as Salmon does citeturn0file0).

3. **Scalability**  
   The above code snippets are geared toward clarity rather than speed or memory efficiency. Real RNA‑seq data sets can contain tens of millions of reads, so you would likely want to:
   - **Collapse or bin reads** that share the same mapping profile (equivalence classes).
   - **Use iterative or streaming algorithms** (like SCVB0 or online EM) rather than a large, all-in-memory assignment matrix.

4. **Connection to Salmon**  
   Salmon’s real implementation is more elaborate: it uses quasi-mapping or alignments, runs an online phase to estimate abundance and bias parameters, and then refines abundance in an offline phase using collapsed EM or VBEM citeturn0file0. The code here is merely a self-contained illustration of the underlying statistical ideas in Python.

Using these building blocks, you can tailor the model details, incorporate advanced bias terms, or switch from SLSQP-based minimization to a hand-rolled EM iteration, depending on what best mirrors the behavior of Salmon or suits your needs for quantification.